# Lab 8, Module 2: Training a Toy Denoiser

**Estimated time:** 18-22 minutes

---

## **Opening: From Destruction to Reconstruction**

In Module 1, you learned the "easy" part: **adding noise** to destroy information.

Now comes the exciting part: **training a model to reverse this process**!

In this module, you'll:
- Build a **simplified U-Net** architecture
- Train it to **predict the noise** added to MNIST digits
- Watch it learn to denoise images in just **2-3 minutes**!
- Visualize learned filters (edge detectors emerge automatically)

**Key insight:** The model doesn't try to directly predict the clean image—it predicts the **noise**, then we subtract it!

---

## 📊 **About the Training Task**

### **What We're Training:**
A **denoising model** that takes two inputs:
1. **Noisy image** (x_t at some timestep t)
2. **Timestep** (t from 0 to 199)

And produces one output:
- **Predicted noise** (ε̂ that was added)

### **Training Process:**
```
1. Pick random clean image from MNIST
2. Pick random timestep t
3. Add noise according to forward diffusion formula
4. Feed noisy image + timestep to model
5. Model predicts the noise
6. Compare predicted noise to actual noise (MSE loss)
7. Update weights via gradient descent
8. Repeat thousands of times!
```

### **Connection to Lab 7:**
Remember training the CNN on MNIST in Lab 7, Module 4?

| Aspect | Lab 7 (CNN) | Lab 8 (Denoiser) |
|--------|-------------|------------------|
| **Task** | Classify digit (0-9) | Predict noise |
| **Input** | Clean image | Noisy image + timestep |
| **Output** | Class probabilities | Noise image |
| **Loss** | Cross-entropy | MSE (mean squared error) |
| **Training** | Gradient descent | Same! |
| **Architecture** | Encoder (downsample) | U-Net (down + up) |

**Key similarity:** Same training process—just different architecture and task!

---

## 📦 **Setup: Import Libraries**


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

print(f"✅ TensorFlow version: {tf.__version__}")
print("✅ Libraries imported successfully!")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Check for GPU
print(f"\n🖥️  GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("   ✓ Training will use GPU (T4) - expect 2-3 minutes")
else:
    print("   ⚠ No GPU detected - training will be slower (~5-10 minutes)")
    print("   💡 In Colab: Runtime → Change runtime type → T4 GPU")

---

## 📥 **Load and Prepare MNIST Data**

We'll use a **subset** of MNIST to keep training fast:
- Default: **Digits 0-3** (~24,000 images)
- Adjustable: Try 2, 4, 6, or all 10 digit classes!

We'll also **downsample to 16×16** to make training faster.


In [ ]:
# USER CONFIGURATION: Adjust this to control dataset size!
NUM_CLASSES = 4  # Options: 2, 4, 6, or 10 (default: 4 for balance of speed and variety)

print(f"Loading MNIST dataset with {NUM_CLASSES} digit classes (0 to {NUM_CLASSES-1})...\n")

# Load MNIST
(x_train_full, y_train_full), (x_test, y_test) = keras.datasets.mnist.load_data()

# Filter to selected classes
mask = y_train_full < NUM_CLASSES
x_train = x_train_full[mask]
y_train = y_train_full[mask]

print(f"✅ Filtered to {len(x_train)} training images")
print(f"   Classes: {np.unique(y_train)}")

# Downsample to 16x16 using bilinear interpolation
x_train_resized = tf.image.resize(x_train[..., np.newaxis], [16, 16]).numpy()

# Normalize to [0, 1]
x_train_resized = x_train_resized.astype('float32') / 255.0

print(f"✅ Downsampled to 16×16 pixels")
print(f"   Final shape: {x_train_resized.shape}")
print(f"   Pixel range: [{x_train_resized.min():.2f}, {x_train_resized.max():.2f}]")

# Display sample from each class
fig, axes = plt.subplots(1, NUM_CLASSES, figsize=(NUM_CLASSES*2, 2))
if NUM_CLASSES == 1:
    axes = [axes]
    
for i in range(NUM_CLASSES):
    idx = np.where(y_train == i)[0][0]
    axes[i].imshow(x_train_resized[idx, :, :, 0], cmap='gray')
    axes[i].set_title(f'Digit {i}', fontsize=12, fontweight='bold')
    axes[i].axis('off')

plt.suptitle(f'Sample 16×16 Digits (Classes 0-{NUM_CLASSES-1})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n💡 Tip: Want to experiment? Change NUM_CLASSES above to 2, 6, or 10!")
print(f"   - 2 classes: ~12k images, trains in ~1-2 min")
print(f"   - 4 classes: ~24k images, trains in ~2-3 min (recommended)")
print(f"   - 6 classes: ~36k images, trains in ~3-4 min")
print(f"   - 10 classes: ~60k images, trains in ~5-6 min")

---

## 🔧 **Setup Diffusion Schedule (From Module 1)**

We'll use the same noise schedule from Module 1.


In [ ]:
# Diffusion hyperparameters
NUM_TIMESTEPS = 200
BETA_START = 0.0001
BETA_END = 0.02

# Create linear beta schedule
betas = np.linspace(BETA_START, BETA_END, NUM_TIMESTEPS).astype('float32')
alphas = 1.0 - betas
alphas_cumprod = np.cumprod(alphas).astype('float32')

# Convert to TensorFlow constants for faster computation
alphas_cumprod_tf = tf.constant(alphas_cumprod, dtype=tf.float32)

print("✅ Diffusion schedule created!")
print(f"   Timesteps: {NUM_TIMESTEPS}")
print(f"   Beta range: [{BETA_START}, {BETA_END}]")
print(f"   Alpha_bar at t=0: {alphas_cumprod[0]:.4f}")
print(f"   Alpha_bar at t=100: {alphas_cumprod[99]:.4f}")
print(f"   Alpha_bar at t=199: {alphas_cumprod[199]:.4f}")

---

## 🏗️ **Build Simplified U-Net Architecture**

### **What is a U-Net?**

A U-Net has two parts:
1. **Encoder (downsampling):** Extract features, reduce spatial size
2. **Decoder (upsampling):** Reconstruct spatial details, increase size
3. **Skip connections:** Copy features from encoder to decoder

**Why U-Net for diffusion?**
- Encoder: Understands what's in the noisy image
- Decoder: Reconstructs the noise pattern
- Skip connections: Preserve spatial details

### **Our Simplified Architecture:**
```
Input: 16×16 noisy image + timestep embedding
    ↓
Encoder Block 1: Conv(32) → Pool → 8×8
    ↓
Encoder Block 2: Conv(64) → Pool → 4×4
    ↓
Bottleneck: Conv(128) → 4×4
    ↓
Decoder Block 1: Upsample → Concat(skip) → Conv(64) → 8×8
    ↓
Decoder Block 2: Upsample → Concat(skip) → Conv(32) → 16×16
    ↓
Output: Conv(1) → 16×16 predicted noise
```

### **Timestep Embedding:**
We need to tell the model "what timestep is this?" because:
- At t=0: Almost no noise (predict tiny corrections)
- At t=100: 50% noise (predict moderate corrections)
- At t=199: Almost all noise (predict large-scale structure)

We'll use **sinusoidal embeddings** (like position encodings in Transformers).

---


In [ ]:
def get_timestep_embedding(timesteps, embedding_dim=32):
    """
    Create sinusoidal timestep embeddings.
    
    Args:
        timesteps: Tensor of timestep indices [batch_size]
        embedding_dim: Dimensionality of embeddings
    
    Returns:
        embeddings: Tensor of shape [batch_size, embedding_dim]
    """
    half_dim = embedding_dim // 2
    emb = np.log(10000) / (half_dim - 1)
    emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
    emb = tf.cast(timesteps, dtype=tf.float32)[:, None] * emb[None, :]
    emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
    return emb

def build_unet_denoiser(image_size=16, channels=1, timestep_dim=32):
    """
    Build a simplified U-Net for noise prediction.
    
    Args:
        image_size: Image size (16x16)
        channels: Number of channels (1 for grayscale)
        timestep_dim: Dimension of timestep embeddings
    
    Returns:
        model: Keras model that takes (noisy_image, timestep) and outputs predicted noise
    """
    # Inputs
    noisy_image_input = layers.Input(shape=(image_size, image_size, channels), name='noisy_image')
    timestep_input = layers.Input(shape=(), dtype=tf.int32, name='timestep')
    
    # Timestep embedding
    t_emb = layers.Lambda(lambda t: get_timestep_embedding(t, timestep_dim))(timestep_input)
    t_emb = layers.Dense(64, activation='relu')(t_emb)
    t_emb = layers.Dense(64, activation='relu')(t_emb)
    
    # Add timestep to image (broadcast across spatial dimensions)
    t_emb_reshaped = layers.Reshape((1, 1, 64))(t_emb)
    t_emb_broadcast = layers.Lambda(lambda x: tf.tile(x, [1, image_size, image_size, 1]))(t_emb_reshaped)
    
    # Concatenate image and timestep
    x = layers.Concatenate()([noisy_image_input, t_emb_broadcast])
    
    # Encoder Block 1: 16x16 → 8x8
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    skip1 = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2)(skip1)
    
    # Encoder Block 2: 8x8 → 4x4
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    skip2 = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2)(skip2)
    
    # Bottleneck: 4x4
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    
    # Decoder Block 1: 4x4 → 8x8
    x = layers.UpSampling2D(2)(x)
    x = layers.Concatenate()([x, skip2])  # Skip connection
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    
    # Decoder Block 2: 8x8 → 16x16
    x = layers.UpSampling2D(2)(x)
    x = layers.Concatenate()([x, skip1])  # Skip connection
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    
    # Output: Predicted noise (same size as input image)
    noise_output = layers.Conv2D(channels, 3, padding='same', name='noise_pred')(x)
    
    # Build model
    model = keras.Model(inputs=[noisy_image_input, timestep_input], outputs=noise_output, name='unet_denoiser')
    
    return model

# Build the model
print("Building U-Net denoiser...\n")
model = build_unet_denoiser(image_size=16, channels=1, timestep_dim=32)

# Print summary
print("📋 Model Architecture:\n")
model.summary()

# Count parameters
total_params = model.count_params()
print(f"\n📊 Total parameters: {total_params:,}")
print("\n💡 This is a simplified U-Net!")
print("   Real diffusion models (Stable Diffusion) have billions of parameters.")
print("   But the core idea is the same!")

---

## 🎯 **Create Training Dataset**

We need to create training pairs:
- Input: (noisy image, timestep)
- Target: actual noise that was added

We'll create these **on-the-fly** during training using a custom data generator.


In [ ]:
def create_training_batch(images, batch_size, alphas_cumprod_tf, num_timesteps):
    """
    Create a training batch by:
    1. Sample random images
    2. Sample random timesteps
    3. Add noise according to forward diffusion
    4. Return (noisy_images, timesteps), noise_targets
    """
    # Sample random images
    indices = np.random.randint(0, len(images), batch_size)
    batch_images = images[indices]
    
    # Sample random timesteps
    timesteps = np.random.randint(0, num_timesteps, batch_size)
    
    # Get alpha_bar for these timesteps
    alpha_bar_t = tf.gather(alphas_cumprod_tf, timesteps)
    
    # Generate random noise
    noise = tf.random.normal(shape=batch_images.shape)
    
    # Apply forward diffusion
    sqrt_alpha_bar_t = tf.sqrt(alpha_bar_t)[:, None, None, None]
    sqrt_one_minus_alpha_bar_t = tf.sqrt(1.0 - alpha_bar_t)[:, None, None, None]
    
    noisy_images = sqrt_alpha_bar_t * batch_images + sqrt_one_minus_alpha_bar_t * noise
    
    return (noisy_images, timesteps), noise

# Test the batch generator
test_batch_inputs, test_batch_targets = create_training_batch(
    x_train_resized, batch_size=4, 
    alphas_cumprod_tf=alphas_cumprod_tf,
    num_timesteps=NUM_TIMESTEPS
)

print("✅ Training batch generator created!")
print(f"\nExample batch:")
print(f"  Noisy images shape: {test_batch_inputs[0].shape}")
print(f"  Timesteps shape: {test_batch_inputs[1].shape}")
print(f"  Noise targets shape: {test_batch_targets.shape}")
print(f"  Example timesteps: {test_batch_inputs[1].numpy()}")

---

## 📝 **Question Q10 (Prediction)**

### **Q10. Before training, predict: How long will training take with your current NUM_CLASSES setting?**

*Consider:*
- **30 seconds** (very fast)
- **2-3 minutes** (fast)
- **10-15 minutes** (moderate)
- **30+ minutes** (slow)

*With GPU (T4), what do you expect?*

**Record your prediction in the Answer Sheet BEFORE continuing!**

---

## 🚀 **Compile and Train the Model!**

Let's train! We'll use:
- **Loss:** MSE (mean squared error) between predicted and actual noise
- **Optimizer:** Adam with learning rate 1e-3
- **Epochs:** 8 (sufficient for convergence)
- **Batch size:** 128


In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss='mse'
)

print("✅ Model compiled!")
print("\nTraining configuration:")
print("  - Optimizer: Adam (lr=1e-3)")
print("  - Loss: MSE (mean squared error)")
print(f"  - Epochs: 8")
print(f"  - Batch size: 128")
print(f"  - Training samples: {len(x_train_resized):,}")
print(f"  - Steps per epoch: {len(x_train_resized) // 128}")

In [ ]:
# Training loop with custom batch generation
EPOCHS = 8
BATCH_SIZE = 128
STEPS_PER_EPOCH = len(x_train_resized) // BATCH_SIZE

print("🚀 Starting training...\n")
print(f"Expected time: ~2-3 minutes with GPU, ~5-10 minutes without\n")
print("="*70)

# Track training history
history = {'loss': []}
start_time = time.time()

for epoch in range(EPOCHS):
    epoch_losses = []
    
    for step in range(STEPS_PER_EPOCH):
        # Generate training batch
        (noisy_images, timesteps), noise_targets = create_training_batch(
            x_train_resized, BATCH_SIZE, alphas_cumprod_tf, NUM_TIMESTEPS
        )
        
        # Train on batch
        loss = model.train_on_batch([noisy_images, timesteps], noise_targets)
        epoch_losses.append(loss)
    
    # Record epoch loss
    epoch_loss = np.mean(epoch_losses)
    history['loss'].append(epoch_loss)
    
    # Print progress
    elapsed = time.time() - start_time
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {epoch_loss:.6f} - Time: {elapsed:.1f}s")

total_time = time.time() - start_time
print("\n" + "="*70)
print(f"✅ Training complete in {total_time:.1f} seconds ({total_time/60:.1f} minutes)!")
print(f"\nFinal loss: {history['loss'][-1]:.6f}")
print(f"Target loss: ~0.05 (lower is better)")

if history['loss'][-1] < 0.1:
    print("\n✓ Good convergence! Model learned to predict noise.")
elif history['loss'][-1] < 0.2:
    print("\n✓ Reasonable convergence. Model is learning.")
else:
    print("\n⚠ Loss is high. Model may need more training.")
    print("   Try increasing EPOCHS or checking GPU availability.")

---

## 📊 **Visualize Training Progress**


In [ ]:
# Plot training loss
plt.figure(figsize=(10, 5))
plt.plot(history['loss'], marker='o', linewidth=2, markersize=8, color='blue')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss (MSE)', fontsize=12)
plt.title('Training Loss: Learning to Predict Noise', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.05, color='green', linestyle='--', linewidth=1, label='Target loss (~0.05)')
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("\n📈 Key observations:")
print(f"  - Initial loss: {history['loss'][0]:.4f} (random predictions)")
print(f"  - Final loss: {history['loss'][-1]:.4f} (learned predictions)")
print(f"  - Improvement: {(1 - history['loss'][-1]/history['loss'][0])*100:.1f}% reduction")
print("\n✓ Loss decreased → Model learned to predict noise!")

---

## 📝 **Questions Q11-Q12 (Observations)**

### **Q11. What final loss did you achieve? Is it close to the target of ~0.05?**

*Look at the final loss value above. Lower is better!*

**Record your answer in the Answer Sheet.**

---

### **Q12. How long did training take? Was it faster or slower than your prediction from Q10?**

*Look at the total training time. Did GPU help?*

**Record your answer in the Answer Sheet.**

---

## 🧪 **Test the Denoiser**

Let's see if the model can actually remove noise!


In [ ]:
# Pick a test image
test_img = x_train_resized[0:1]  # Shape: (1, 16, 16, 1)

# Test at different timesteps
test_timesteps = [25, 50, 100, 150]

fig, axes = plt.subplots(len(test_timesteps), 5, figsize=(16, len(test_timesteps)*3))

for i, t in enumerate(test_timesteps):
    # Get alpha_bar for this timestep
    alpha_bar_t = alphas_cumprod[t]
    
    # Add noise
    noise = np.random.randn(*test_img.shape)
    sqrt_alpha_bar_t = np.sqrt(alpha_bar_t)
    sqrt_one_minus_alpha_bar_t = np.sqrt(1.0 - alpha_bar_t)
    noisy = sqrt_alpha_bar_t * test_img + sqrt_one_minus_alpha_bar_t * noise
    
    # Predict noise
    timestep_input = np.array([t])
    predicted_noise = model.predict([noisy, timestep_input], verbose=0)
    
    # Denoise
    denoised = (noisy - sqrt_one_minus_alpha_bar_t * predicted_noise) / sqrt_alpha_bar_t
    
    # Plot
    axes[i, 0].imshow(test_img[0, :, :, 0], cmap='gray', vmin=0, vmax=1)
    axes[i, 0].set_title('Original', fontsize=11, fontweight='bold')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(noisy[0, :, :, 0], cmap='gray')
    axes[i, 1].set_title(f'Noisy (t={t})', fontsize=11, fontweight='bold')
    axes[i, 1].axis('off')
    
    axes[i, 2].imshow(noise[0, :, :, 0], cmap='RdBu', vmin=-2, vmax=2)
    axes[i, 2].set_title('Actual Noise', fontsize=11, fontweight='bold')
    axes[i, 2].axis('off')
    
    axes[i, 3].imshow(predicted_noise[0, :, :, 0], cmap='RdBu', vmin=-2, vmax=2)
    axes[i, 3].set_title('Predicted Noise', fontsize=11, fontweight='bold')
    axes[i, 3].axis('off')
    
    axes[i, 4].imshow(np.clip(denoised[0, :, :, 0], 0, 1), cmap='gray', vmin=0, vmax=1)
    axes[i, 4].set_title('Denoised', fontsize=11, fontweight='bold')
    axes[i, 4].axis('off')

plt.tight_layout()
plt.suptitle('Denoising Performance at Different Timesteps', fontsize=15, fontweight='bold', y=1.01)
plt.show()

print("\n💡 Observations:")
print("  - At t=25 (low noise): Model should denoise well")
print("  - At t=100 (medium noise): Model should partially denoise")
print("  - At t=150+ (high noise): Harder to recover structure")
print("\n✓ The model learned to predict noise at different noise levels!")

---

## 📝 **Questions Q13-Q15 (Analysis)**

### **Q13. At t=100, does the denoiser successfully remove noise? How does the denoised image compare to the original?**

*Look at the middle row. Is the digit recognizable after denoising?*

**Record your answer in the Answer Sheet.**

---

### **Q14. Compare denoising performance at t=25 vs t=150. At which timestep does the model perform better? Why?**

*Hint: Less noise = easier to denoise. More signal remains at lower timesteps.*

**Record your answer in the Answer Sheet.**

---

### **Q15. How is this similar to Lab 7, Module 4 where you trained a CNN on MNIST? How is it different?**

*Think about: training process, architecture, task, loss function*

**Record your answer in the Answer Sheet.**

---

## 🔬 **Visualize Learned Filters**

Just like in Lab 7, let's see what the first convolutional layer learned!


In [ ]:
# Get first conv layer weights
first_conv_layer = None
for layer in model.layers:
    if isinstance(layer, layers.Conv2D):
        first_conv_layer = layer
        break

if first_conv_layer is not None:
    weights = first_conv_layer.get_weights()[0]  # Shape: (3, 3, input_channels, 32)
    num_filters = min(32, weights.shape[-1])
    
    fig, axes = plt.subplots(4, 8, figsize=(16, 8))
    axes = axes.flatten()
    
    for i in range(num_filters):
        filter_img = weights[:, :, 0, i]  # Get first input channel
        axes[i].imshow(filter_img, cmap='RdBu')
        axes[i].set_title(f'Filter {i+1}', fontsize=9)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle('Learned Filters: Do They Look Like Edge Detectors?', 
                 fontsize=14, fontweight='bold', y=1.01)
    plt.show()
    
    print("\n💡 Look for:")
    print("  - Edge detectors (vertical, horizontal, diagonal)")
    print("  - Gradient patterns")
    print("  - Similar to Sobel filters from Lab 7, Module 1!")
    print("\n✓ The model learned useful features automatically through training!")
else:
    print("Could not find convolutional layer")

---

## ✅ Module 2 Complete!

You now understand:
- **U-Net architecture** (encoder-decoder with skip connections)
- **Training a denoiser** (predict noise, not the image directly)
- **Timestep embeddings** (model needs to know "how noisy is this?")
- **Fast training** (2-3 minutes on GPU for toy model)
- **Learned features** (automatic edge detector emergence)
- **Denoising performance** (better at low noise, harder at high noise)

**Key insights:**
1. **Same training process as Lab 4 and Lab 7** - gradient descent works!
2. **Model predicts noise, not images** - indirect approach is easier to learn
3. **Timestep matters** - model needs context about noise level
4. **U-Net preserves spatial details** - skip connections help

**Ready to generate images?**

Move on to **Module 3: Multi-Step Denoising**, where you'll use this trained model to generate digits from pure noise!

---